# Generate forcings

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/IHCantabria/pyteseo/blob/main/docs/notebooks/04_create_forcings.ipynb)

## Install pyTESEO
- If you are running this notebook from `Google colab` uncomment and run `!pip install pyteseo && pyteseo-tests` in the cell below. 
Doing this you will install `pyteseo` from `pypi` repository and run `pyteseo-tests` to check the installation (all tests have to succeed).

- If you are using this notebook in your own local machine you need to install the library from the local repo `pip install .` or pypi `pip install pyteseo`.

In [ ]:
# !pip install pyteseo && pyteseo-tests

In [ ]:
import xarray as xr
from datetime import datetime, timedelta
from pyteseo.io.forcings import write_2d_forcing
from pyteseo.wrapper import TeseoWrapper
from pathlib import Path

In [ ]:
# COPERNICUS OPENDAP ACCESS
!pip install pydap


# Funcion to save Dataset object in multiple netcdf files by day
def to_daily_netcdf(ds: xr.Dataset, output_path: str, netcdf_format: str = None):
    from pathlib import Path

    output_path = str(Path(output_path).resolve())
    output_dir = str(Path(output_path).parent)
    filename, file_ext = str(Path(output_path).stem), str(Path(output_path).suffix)

    date, datasets = zip(*ds.groupby("time.date"))
    paths = [f"{output_dir}/{filename}_{d}{file_ext}" for d in date]
    xr.save_mfdataset(datasets, paths, format=netcdf_format)
    return paths

In [ ]:
def generate_forcing(
    lon_min, lon_max, lat_min, lat_max, t_min, t_max, dataset_id, dir_path, forcing_type
):

    longitudes = slice(lon_min, lon_max)
    latitudes = slice(lat_min, lat_max)
    times = slice(t_min, t_max)

    data_store = _copernicusmarine_datastore(dataset_id, username, password)
    ds = xr.open_dataset(data_store)

    ds_subset = ds.sel(
        longitude=longitudes,
        latitude=latitudes,
        time=times,
        method="nearest",
    )

    write_2d_forcing(ds_subset, dir_path, forcing_type)

In [ ]:
dir_path = Path("test_simulation")

job = TeseoWrapper(dir_path)

lon_min = -5.5
lon_max = -1
lat_min = 43.25
lat_max = 44.25

t_min = datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0)
t_max = t_min + timedelta(days=5)

global_cmems_datasets = [
    {
        "dataset_id": "cmems_mod_glo_phy_anfc_merged-uv_PT1H-i",
        "dataset_type": "currents",
    },
    {
        "dataset_id": "cmems_obs-wind_glo_phy_nrt_l4_0.125deg_PT1H",
        "dataset_type": "winds",
    },
]


for dataset in global_cmems_datasets:
    generate_forcing(
        lon_min,
        lon_max,
        lat_min,
        lat_max,
        t_min,
        t_max,
        dataset["dataset_id"],
        dir_path,
        dataset["dataset_type"],
    )

job.load_inputs()